In [2]:
class EmailConfigApp:
    
    def __init__(self, root):
        self.root = root
        self.root.title("MoonBack Email System")
        self.root.geometry("680x550")

        self.fields = [
               ("SMTP Server:", "smtp.gmail.com"),
               ("SMTP Port:", "587"),
               ("Sender Name:", "xxx"),
               ("Sender Email:", "xxx@gmail.com"),
               ("Password:", "xxx"),
               ("Excel File:", ".\\influencers.xlsx"),
               ("Picture Catalog:", "")
           ]

        self.create_widgets()
        self.sending = False

    def create_widgets(self):

        frame_input = ttk.LabelFrame(self.root, text="Please input the following information:")
        # Place the frame_input widget in the grid system, specifying its position and layout parameters
        # row=0: Place the frameinput widget in the first row of the grid.
        # column=0: Place the frameinput widget in the first column of the grid.
        # padx=10: Add a padding of 10 pixels on the left and right sides of the frameinput widget.
        # pady=5: Add a padding of 5 pixels on the top and bottom sides of the frameinput widget.
        # sticky="nsew": Make the frameinput widget stick to the north, south, east, and west borders of its cell, allowing it to expand in all directions if the window is resized.
        frame_input.grid(row=0, column=0, padx=10, pady=5, sticky="nsew")

        self.entries = {}

        for i, (label, default) in enumerate(self.fields):

            ttk.Label(frame_input, text=label).grid(row=i, column=0, padx=5, pady=5, sticky='w')

            entry = ttk.Entry(frame_input, width=40)

            entry.insert(0, default)

            if "Password" in label:
                entry.config(show="*")

            entry.grid(row=i, column=1, padx=5, pady=5)

            if any(x in label for x in ["Excel File"]):

                ttk.Button(frame_input, text="Search", command=lambda e=entry: self.browse_path_1(e)).grid(row=i, column=2)

            if any(x in label for x in ["Picture Catalog"]):

                ttk.Button(frame_input, text="Search", command=lambda e=entry: self.browse_path_2(e)).grid(row=i, column=2)

            self.entries[label.strip(":")] = entry
            
        frame_content = ttk.LabelFrame(self.root, text="Email Content")

        # Place the frame_content widget in the grid at row index 2 and column index 0.
        # Add a padding of 10 pixels on the left and right sides (padx=10) and 5 pixels on the top and bottom sides (pady=5).
        # Make the frame_content widget stick to the north, south, east, and west borders of its cell (sticky="nsew"),
        # which allows it to expand in all directions if the window is resized.
        frame_content.grid(row=2, column=0, padx=10, pady=5, sticky="nsew")
        frame_content.grid_rowconfigure(0, weight=1)
        frame_content.grid_columnconfigure(0, weight=1)

        # Create a Text widget inside the frame_content widget, which is used for multi-line text input.
        # The text will wrap at word boundaries (wrap=tk.WORD) and the widget will have a height of 20 lines.
        self.mail_content = tk.Text(frame_content, wrap=tk.WORD, height=20)
        self.mail_content.grid(row=0, column=0, sticky="nsew")

        # Create a Scrollbar widget inside the frame_content widget.
        # The scrollbar is oriented vertically (orient=tk.VERTICAL) and is linked to the y-axis view of the self.mail_content Text widget
        # via the command=self.mail_content.yview, which allows scrolling the text content.
        scrollbar = ttk.Scrollbar(frame_content, orient=tk.VERTICAL, command=self.mail_content.yview)
        scrollbar.grid(row=0, column=1, sticky="ns")

        self.mail_content.configure(yscrollcommand=scrollbar.set)

        self.default_template = """<html>
        <body style="font-family: Microsoft YaHei; font-size: 14px;">
        Hi {nickname},<br><br>

        I love your content about doggie!  We’re looking for passionate pet creators to join our TikTok Shop program!<br>
        You can pick a plush dog toy from our shop to start promoting it. If we work well together, we’ll send you the full set for free!<br><br>

        Here’s why you’ll love working with us:<br> 
        ✅ High commissions on every sale<br> 
        ✅ Over 2,000 five-star reviews on Amazon<br>
        ✅ Consistently ranks in the Top 5 of Amazon’s "Dog Mental Stimulation Toys" category<br>
        ✅ Fun and engaging pet products your audience will love<br> 
        ✅ Long-term brand partnership opportunities<br><br>

        <strong>Note:</strong><br>
        <div style="margin-left:15px;">
        1. Shop Name: <em>Pet Enrichment Toys</em><br>
        2. Product Link: 
        <a href="https://xxx">View Collection</a><br>
        </div><br>

        Looking forward to your early reply!<br><br>

        Sincerely,<br>
        xxx
        </body>
        </html>"""

        self.mail_content.insert("1.0", self.default_template)


        frame_subject = ttk.LabelFrame(self.root, text="Subject")

        frame_subject.grid(row=3, column=0, padx=10, pady=5, sticky="ew")

        self.mail_subject = ttk.Entry(frame_subject, width=60)

        self.mail_subject.grid(row=0, column=0, padx=5, pady=3, sticky="w")

        self.default_subject = "Invitation for {nickname} to Join TikTok Program 🐾" 

        self.mail_subject.insert(0, self.default_subject)

        self.mail_subject.config(foreground='#666')


        self.log_area = tk.Text(self.root, height=12, state='disabled')

        self.log_area.grid(row=4, column=0, padx=10, pady=5, sticky="nsew")


        btn_frame = ttk.Frame(self.root)

        btn_frame.grid(row=4, column=0, pady=10)

        ttk.Button(btn_frame, text="Start Sending", command=self.start_sending).pack(side=tk.LEFT, padx=5)

        ttk.Button(btn_frame, text="Stop Sending", command=self.stop_sending).pack(side=tk.LEFT, padx=5)

        self.root.grid_rowconfigure(0, weight=0)
        self.root.grid_rowconfigure(1, weight=1)  
        self.root.grid_rowconfigure(2, weight=0)
        self.root.grid_rowconfigure(3, weight=0)
        self.root.grid_columnconfigure(0, weight=1)

    def clear_placeholder(self, event):

        current_content = self.mail_content.get("1.0", "end-1c")

        if current_content == self.default_template:

            self.mail_content.delete("1.0", tk.END)

            self.mail_content.config(fg='black')

    def browse_path_1(self, entry):

        desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")


        file_path = filedialog.askopenfilename(
                title="Please select the excel file",
                initialdir=desktop_path,  
                filetypes=[
                    ("Excel", "*.xlsx"), 
                    ("Excel 97-2003", "*.xls"),
                    ("All Files", "*.*")
                ]  
            )

        if file_path:
            entry.delete(0, tk.END)
            entry.insert(0, file_path)

    def browse_path_2(self, entry):

        desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")

        file_path = filedialog.askdirectory(
            initialdir=desktop_path, 
            title="Please select the photo catalog"
        )

        if file_path:
            entry.delete(0, tk.END)
            entry.insert(0, file_path)

    def log(self, message):

        self.log_area.config(state='normal')

        self.log_area.insert(tk.END, message + "\n") 

        self.log_area.see(tk.END)

        self.log_area.config(state='disabled')

    def start_sending(self):

        if not self.sending:

            self.sending = True

            config = {k: v.get() for k, v in self.entries.items()}

            self.log("Initializing...")

            threading.Thread(target=self.run_sending, args=(config,)).start()

    def stop_sending(self):

        self.sending = False

        self.log("Terminating...")
                                
    def run_sending(self, config):
       
        try:
                                
            if not os.path.exists(config["Excel File"]):
                messagebox.showerror("Error", "Excel File do not exist！")
                return
                
            df = pd.read_excel(config["Excel File"])
            df_clean = df.dropna(subset=["nickname", "receiver email"])
            df_clean = df_clean.applymap(lambda x: x.strip() if isinstance(x, str) else x)

            server = smtplib.SMTP(config["SMTP Server"], int(config["SMTP Port"]))
            server.ehlo()
            server.starttls()
            server.login(config["Sender Email"], config["Password"])
            self.log("[SMTP] Server Sucessfully Connected.")

            total = len(df_clean)
            for idx, (_, row) in enumerate(df_clean.iterrows(), 1):
                if not self.sending: break
                
                nickname = row["nickname"]
                receiver_email = row["receiver email"]
                msg = self.build_email(config, nickname, receiver_email)
                
                try:
                    server.sendmail(config["Sender Email"], [receiver_email], msg.as_string())
                    self.log(f"[{idx}/{total}] ✅ Sent successfully：{nickname} <{receiver_email}>")
                    time.sleep(12 + random.randint(0, 3))
                except Exception as e:
                    self.log(f"❌ Send failure：{receiver_email}，Error：{str(e)}")

            server.quit()
            self.log("The email sending task is complete")
        except Exception as e:
            self.log(f"⚠️ System error：{str(e)}")
        finally:
            self.sending = False
                                
    def build_email(self, config, nickname, receiver_email):
        
        msg = MIMEMultipart('related')

        email_template = self.mail_content.get("1.0", tk.END).strip()
        
        if not email_template:
            self.log("❌ The email content cannot be empty.")
        raise ValueError("The email content is empty.")
                                
        product_images = ""
        image_dir = config["Picture Catalog"]
                                
        for i in range(1, 4):
            img_path = os.path.join(image_dir, f"image_{i}.jpg")
            try:
                 with open(img_path, 'rb') as f:
                     img = MIMEImage(f.read(), _subtype='jpeg')
                 img.add_header('Content-ID', f'<image{i}>')
                 msg.attach(img)
                 product_images += f'<img src="cid:image{i}" style="max-width:600px;"><br>'
            except Exception as e:
                 self.log(f"⚠️ Image loading failure：{img_path}")
                                
        email_body = email_template.format(
            nickname=nickname,
            product_images=product_images
        )
       
        msg.attach(MIMEText(email_body, 'html', 'utf-8'))
        
        msg["From"] = formataddr((config["Sender Name"], config["Sender Email"]))
        msg["To"] = formataddr((nickname, receiver_email))
        msg["Subject"] = self.mail_subject.get("1.0", tk.END).strip()

        return msg